In [1]:
import sys
sys.path.insert(0, 'caffe/python')
import caffe
import numpy as np
import scipy.misc
import cv2
import scipy.io
import os, sys, argparse
from os.path import join, splitext, split, isfile
import pylayer

In [2]:
caffe.set_mode_gpu()

def convert(data):
    assert data.ndim == 3
    data -= np.array((104.00698793,116.66876762,122.67891434))
    data = data.transpose((2, 0, 1))
    return data

def inverse_convert(data):
    assert data.ndim == 3
    data = data.transpose((1, 2, 0))
    data += np.array((104.00698793,116.66876762,122.67891434))
    return data

def rescale_image(image):
    return image / 255.
    
def convert_label(label):
    mask = label >= 25
    label[mask] = 1
    label[~mask] = 0
    return label

model = 'model/hed_pretrained_bsds.caffemodel'


test_dir = '../deeplab/models/research/deeplab/datasets/pascal_voc_seg/VOCdevkit/VOC2012/JPEGImages_original' # test images directory




In [3]:
def get_image(image_name):
    
    img = cv2.cvtColor(cv2.imread(image_name), cv2.COLOR_BGR2RGB).astype(np.float32)
    return convert(img)
def get_label(label_name):
    img = cv2.imread(label_name).astype(np.float32)
    return convert_label(np.squeeze(img[:,:,:1]))

diversity_prob = 0.0
epsilon = 16
step = 2
mom = 0.5
smoothing = 0


In [4]:

def forward_test(dat):
    net.blobs['data'].reshape(1, *dat.shape)
    net.blobs['data'].data[...] = dat
    net.forward()
    return np.array(net.blobs['sigmoid_fuse'].data)

argnet = 'model/hed_test.pt'

net = caffe.Net(argnet, model, caffe.TEST)



In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import scipy

for image in os.listdir(test_dir):
    pixels = get_image(os.path.join(test_dir, image))
    prefix = image.split('.')[0]
    try:
        os.makedirs('pascal-hed-outputs')
    except:
        pass
    scipy.misc.imsave(os.path.join('pascal-hed-outputs', prefix + '.png'), np.squeeze(forward_test(pixels)))

In [5]:

argnet = 'model/hed_train_attack.pt'


net = caffe.Net(argnet, model, caffe.TRAIN)


In [ ]:
from hed_attack import attack

targ = False
suppress = False
attacks_dir = './pascal_attacked_{:0.1f}_{:0.1f}_{:0.1f}_{:0.1f}_{}_{}'.format(diversity_prob, epsilon, step, mom, targ, suppress)
try:
    os.makedirs(attacks_dir)
except OSError as e:
    pass

i = 0
for image in os.listdir(test_dir):
    prefix = image.split('.')[0]
    gt = get_label(os.path.join('pascal-hed-outputs', prefix + '.png'))
    pixels = get_image(os.path.join(test_dir, image))
    if suppress:
        att = attack(net, pixels, np.zeros_like(gt), step, epsilon, mom, 10, smoothing, True)
    else:
        att = attack(net, pixels, gt, step, epsilon, mom, 10, smoothing, targ)
    try:
        os.makedirs(attack_dir)
    except:
        pass
    scipy.misc.imsave(join(attacks_dir, prefix + '.png'), att.transpose(1, 2, 0))
    i+=1
    print(i)

Targeted attack transferability experiment

In [ ]:

with open('./map_clsloc.txt') as f:
    dir_to_class = dict([(x[0], x[1]) for x in (y.split() for y in f.read().splitlines())])


In [ ]:
import random
targ = True
attacks_dir = './imagenet_attacked_{:0.1f}_{:0.1f}_{:0.1f}_{:0.1f}_{}'.format(diversity_prob, epsilon, step, mom, targ)
try:
    os.makedirs(attacks_dir)
except OSError as e:
    pass
real_dir = './output'

i = 0
for folder in os.listdir(test_dir):
    for image in os.listdir(os.path.join(test_dir, folder)):
        # get a random edge output
        c = random.choice(os.listdir('imagenet-hed-outputs'))
        im = random.choice(os.listdir(os.path.join('imagenet-hed-outputs', c)))
        
        gt = get_label(os.path.join('imagenet-hed-outputs', folder, image))
        pixels = get_image(os.path.join(test_dir, folder, image))
        att = attack(pixels, gt, targ)
        try:
            os.makedirs(join(attacks_dir, folder))
        except:
            pass
        scipy.misc.imsave(join(attacks_dir, folder, dir_to_class[c] + '_' + image), att.transpose(1, 2, 0))
        i+=1
    print(i)

In [ ]:
from shutil import copyfile

new_dir = './imagenet_shuffled_{:0.1f}_{:0.1f}_{:0.1f}_{:0.1f}_{}_{}'.format(diversity_prob, epsilon, step, mom, targ, smoothing)
try:
    os.makedirs(attacks_dir)
except OSError as e:
    pass

for folder in os.listdir(attacks_dir):
    for image in os.listdir(os.path.join(attacks_dir, folder)):
        # get a random edge output
        new_folder = "%04d" % int(image.split('.')[0].split('_')[0])
        
        try:
            os.makedirs(os.path.join(new_dir, new_folder))
        except OSError as e:
            pass
        copyfile(os.path.join(attacks_dir,folder,image), os.path.join(new_dir, new_folder, image))
    print(i)

In [ ]:
sorted([int(x[1]) for x in dir_to_class.iteritems()])